In [14]:
################################################################################
# This program is for Coding Exercise 4    
# 
# Builds off the Barkley code. From that, I have removed the "S" variable that
# allowed for buses of different types (simple=better). I also have rewritten 
# all the code in R such that it follows the algorithms outlined in the 
# Gretchen Sileo lecture slides. There the main difference is that Sileo 
# expresses things mainly with conditional value functions whereas the Barkley 
# code works mainly with unconditional (ex ante) value functions. Also, in 
# implementing CCP estimation, Barkley applies the newer IV approach of 
# Kalouptsidi et al (2021), where I have stuck with a Hotz-Miller approach.
#
# Estimation recovers the structural parameters used to construct the data.
#
# NHM 3-18-2023
#
# Big assists from Ryan Mansley, Tianshi Mu, and Gretchen Sileo
# 
################################################################################

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(12)

# Set working directory
#path <- "C:/Users/nhm27/Dropbox/Teaching Materials/ECON 631 (Empirical IO)/2023/Coding Exercise 4"
path = "/Users/pranjal/Desktop/Structural-Economics/industrial-organization/7_dynamic_discrete_choice"

import os
os.chdir(path)

In [15]:
#########################################################################
# Defining the states: mileage
# - x_min and x_max give the min and max mileage is 0 and the max is 15
# - delta_x is for discretizing the space
# - x_len gives the number of discrete states, which is 301
# - x is a vector of the states
#########################################################################

x_min = 0.0
x_max = 15.0
delta_x = 0.05
x_len = int((x_max - x_min) / delta_x + 1)
x = np.arange(x_min, x_max + delta_x, delta_x)

In [16]:
#########################################################################
# Defining the state transitions
# - F1, F2 has starting state as row i ; ending state as column k
# - Builds in assumptions on how these transitions occur
# - F2 is the transition that occurs **without** replacement
# - F1 is the transition that occurs **with** replacement and is simpler
# - F2b is like F2 but it is the *CDF* not the PDF of the transition distribution
# - Saving Euler's constant for now
#########################################################################

x_tday = np.repeat(x, x_len).reshape(x_len, x_len)
x_next = x_tday.T
f = (x_next >= x_tday) * np.exp(- (x_next - x_tday)) * (1 - np.exp(-delta_x))
f = f.T
f[:, -1] = 1 - np.sum(f[:, :-1], axis=1)
F2 = f
F2b = np.apply_along_axis(np.cumsum, 1, f)
F1 = np.zeros((F2.shape[0], F2.shape[1]))
F1[:, 0] = 1

In [17]:
#########################################################################
# Parameterization
# - beta: Discount factor 
# - theta: theta[1] = base utility of "not repair" (+), 
#          theta[2] = utility of "not repair" decreases with mileage (-) 
# - T: let this play out over T time periods
# - N: number of buses 
#########################################################################

from numpy import random as rnd

beta = 0.9
theta = np.array([2.00, -0.15])
T = 30
X = x
N = 2000

In [18]:
################################################################################
# This function performs a value function contraction mapping.
#
# Inputs include:
#   - flow: matrix with flow payoffs for each mileage (row) and action (column)
#   - F1: mileage transition if action=1 (repair)
#   - F2: mileage transition if action=2 (don't repair)
#   - X: possible mileage
#   - beta: discount factor
#
# Outputs include:
#   - condval: conditional values that satisfy contraction mapping
#   - iter: number of iterations done in contraction (curiosity)
#   - ccp: implied choice probability for action=1 (repair)
#
# This is used for FIML estimation and data generation
################################################################################

import numpy as np

def valuemap(flow, F1, F2, X, beta):
    
    # Candidate value conditional value functions: rows are state; columns are choice
    v0 = np.zeros((len(X), 2))

    # Euler's constant. Not needed but allows condition value function to have
    # same units as the ex ante value function of Hotz-Miller so adding for 
    # better comparability 
    euler = 0.5772
  
    # Solving the contraction mapping
    value_diff = 1
    iter = 0
    while (value_diff > 1e-5) and (iter < 1000):
    
        # Implied inclusive value given state 
        ival = np.log(np.sum(np.exp(v0), axis=1)) + euler
    
        # Implied expectation of next period's value, by decision
        expval = np.concatenate((F1.dot(ival.reshape(-1, 1)), F2.dot(ival.reshape(-1, 1))), axis=1)
    
        # Implied conditional values
        v1 = flow + beta * expval
    
        # Maximum gap
        value_diff = np.max(np.abs(v0 - v1))
    
        # Update
        v0 = v1
        iter += 1
        
    if value_diff > 1e-5:
        print("Contraction Mapping Fails")
        
    # Conditional Choice Probability (CCP): probability that choose repair (d=1)
    ccp = 1 / (1 + np.exp(v0[:,1] - v0[:,0]))
  
    return {'condval': v0, 'iter': iter, 'ccp': ccp}

In [19]:
# test
flow = np.column_stack((np.repeat(0, len(X)), theta[0] + theta[1] * X))
condval = valuemap(flow=flow, F1=F1, F2=F2, X=X, beta=beta)['condval']
ccp = valuemap(flow=flow, F1=F1, F2=F2, X=X, beta=beta)['ccp']
print(condval[0:10])

[[17.4468588  18.51207096]
 [17.4468588  18.5514531 ]
 [17.4468588  18.58981969]
 [17.4468588  18.62719894]
 [17.4468588  18.66361751]
 [17.4468588  18.69910056]
 [17.4468588  18.73367192]
 [17.4468588  18.76735413]
 [17.4468588  18.80016859]
 [17.4468588  18.83213555]]


In [21]:

################################################################################
# This function generates a sample of N buses observed over T years
#
# Inputs include:
#   - F1: mileage transition if action=1 (repair)
#   - F2: mileage transition if action=2 (don't repair)
#   - F2b: a CDF version of F2
#   - X: possible mileage
#   - theta: payoff parameters
#   - beta: discount factor
#
# Outputs include:
#   - data_x: mileages for buses (rows) and periods (columns)
#   - data_d: actions for buses (rows) and periods (columns); =1 for repair, =2 otherwise
#   - data_x_index: useful index
#
# This provides the data used in estimation
################################################################################

def generate_data(N, T, F1, F2, F2b, X, theta, beta):
    initial_run = 5
    x_index = np.ones((N, T + initial_run), dtype=int)

    # Decisions and mileage to be filled in
    d_sim = np.zeros((N, T + initial_run), dtype=int)
    x_sim = np.zeros((N, T + initial_run))

    # Random draws to translate probabilities into decisions and mileage transition
    draw_d = np.random.uniform(size=(N, T + initial_run))
    draw_x = np.random.uniform(size=(N, T + initial_run))

    # Flow utility.
    # - rows are state; columns are choice (1=repair, 2=no repair)
    flow = np.column_stack((np.zeros(len(X)), theta[0] + theta[1] * X))

    # Obtaining conditional value function and implied CCP.
    # - rows are state; columns are choice (1=repair, 2=no repair)
    solution = valuemap(flow=flow, F1=F1, F2=F2, X=X, beta=beta)
    condval = solution['condval']
    ccp = solution['ccp']

    for n in range(N):
        for t in range(T + initial_run - 1):
            # Probability of "repair" and "don't repair"
            p1 = ccp[x_index[n, t]]

            # Use random draws; =1 if "repair" and =2 if "don't repair"
            d_sim[n, t] = 1 if draw_d[n, t] > p1 else 2

            # If don't repair, transition to a new mileage state based on random draw
            x_index[n, t + 1] = sum(draw_x[n, t] > F2b[x_index[n, t], :])


In [23]:
#########################################################################
# Generating data and plotting for Bus 1 for confirmation
# - N buses observed over T periods each 
# - Check that average CCP corresponds to average decision
# - Plot data for Bus 1, t=1,...15, to see whether it is reasonable
#########################################################################

busdata = generate_data(N=N, T=T, F1=F1, F2=F2, F2b=F2b, X=X, theta=theta, beta=beta)

data_t = busdata['data_t']
data_x = busdata['data_x']
data_d = busdata['data_d']
data_x_index = busdata['data_x_index']


# Average CCP vs. empirical repair percentage [EPR = -(ave-2)) ]
# These line up ... **must exclude t=T as no decision is recorded there**
emp_ccp = ccp[data_x_index].reshape(-1, ncol(data_x_index))
ave_ccp = np.mean(emp_ccp[:, :-1])
erp = -(np.mean(data_d[:, :-1])-2)
print([ave_ccp, erp])


# Plot the data for Bus 1
data_to_plot = pd.DataFrame({
    'Time': data_t[0, 0:15],
    'Mileage': data_x[0, 0:15],
    'Replacement': (data_x[0, 0:15] == 0)
})

TypeError: 'NoneType' object is not subscriptable